In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd
import numpy as np

df = pd.read_csv('titanic_reviews.csv')

print("Размер данных")
print(df.shape)

print("Информация")
print(df.info())

print("Строки")
print(df.head(5))

Размер данных
(10, 2)
Информация
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   review          10 non-null     object
 1   true_sentiment  10 non-null     object
dtypes: object(2)
memory usage: 292.0+ bytes
None
Строки
                                              review true_sentiment
0  Этот фильм просто шедевр! Ди Каприо и Уинслет ...       positive
1  Отличный фильм, но слишком затянутый. Можно бы...       positive
2  Не понимаю всей этой шумихи. Обычная мелодрама...       negative
3  Гениальное кино! Смотрел 5 раз и каждый раз пл...       positive
4  Хороший фильм, но не шедевр. Спецэффекты на ур...        neutral


In [ ]:
print("Загрузка модели...")
classifier = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment"
)

Загрузка модели...


c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 201/201 [00:00<00:00, 536.93it/s,

In [14]:
print("Результаты анадиза тональности")

results = []
for i, row in df.iterrows():
    pred = classifier(row['review'])[0]
    
    stars = int(pred['label'].split()[0])
    if stars <= 2:
        sentiment = 'negative'
    elif stars == 3:
        sentiment = 'neutral'
    else:
        sentiment = 'positive'
    
    results.append({
        'review': row['review'][:60] + '...',
        'sentiment': sentiment,
        'confidence': pred['score'],
        'stars': stars,
        'true': row['true_sentiment']
    })
    
    print(f"\nОтзыв {i+1}: {row['review'][:60]}...")
    print(f"   Предсказано: {sentiment} (звезд: {stars}, уверенность: {pred['score']:.3f})")
    print(f"   Реальность:  {row['true_sentiment']}")

correct = sum([r['sentiment'] == r['true'] for r in results])
accuracy = correct / len(results)


Результаты анадиза тональности

Отзыв 1: Этот фильм просто шедевр! Ди Каприо и Уинслет сыграли велико...
   Предсказано: positive (звезд: 5, уверенность: 0.803)
   Реальность:  positive

Отзыв 2: Отличный фильм, но слишком затянутый. Можно было сократить н...
   Предсказано: neutral (звезд: 3, уверенность: 0.681)
   Реальность:  positive

Отзыв 3: Не понимаю всей этой шумихи. Обычная мелодрама, ничего особе...
   Предсказано: negative (звезд: 1, уверенность: 0.437)
   Реальность:  negative

Отзыв 4: Гениальное кино! Смотрел 5 раз и каждый раз плачу. Лучший фи...
   Предсказано: positive (звезд: 5, уверенность: 0.984)
   Реальность:  positive

Отзыв 5: Хороший фильм, но не шедевр. Спецэффекты на уровне, игра акт...
   Предсказано: neutral (звезд: 3, уверенность: 0.634)
   Реальность:  neutral

Отзыв 6: Скукотища! Три часа тонущего корабля и сопливых разговоров. ...
   Предсказано: positive (звезд: 5, уверенность: 0.596)
   Реальность:  negative

Отзыв 7: Прекрасный фильм о любви и самоп

In [15]:
print(f"Итоговая точность {accuracy:.2%} ({correct}/{len(results)})")
results_df = pd.DataFrame(results)
print("\nТаблица результатов:")
print(results_df[['review', 'sentiment', 'confidence']].to_string())

Итоговая точность 70.00% (7/10)

Таблица результатов:
                                                            review sentiment  confidence
0  Этот фильм просто шедевр! Ди Каприо и Уинслет сыграли велико...  positive    0.803408
1  Отличный фильм, но слишком затянутый. Можно было сократить н...   neutral    0.681124
2  Не понимаю всей этой шумихи. Обычная мелодрама, ничего особе...  negative    0.436776
3  Гениальное кино! Смотрел 5 раз и каждый раз плачу. Лучший фи...  positive    0.984376
4  Хороший фильм, но не шедевр. Спецэффекты на уровне, игра акт...   neutral    0.633994
5  Скукотища! Три часа тонущего корабля и сопливых разговоров. ...  positive    0.596248
6  Прекрасный фильм о любви и самопожертвовании. Очень трогател...  positive    0.723750
7  Неплохо, но могло быть лучше. Ди Каприо отличный, но фильм з...   neutral    0.601713
8  Шикарный фильм! Лео гениален! Смотрела в кинотеатре 3 раза. ...  positive    0.525444
9      Обычный фильм-катастрофа. Ничего выдающегося. Про